### Prototype data Precessing with Spacy

1. Import model
2. Import database
3. iterate through database and label the data
4. Create DF with labeled data as features
5. Export DF as 'results.csv'

In [1]:
import spacy
import pandas as pd 
import numpy as np 

In [2]:
# Loading the Model

nlp = spacy.load("../training/test_model")

In [3]:
# Loading the input_data

data = pd.read_csv("input_data_filtered.csv")

In [4]:
data["PrimaryOutcomeMeasure"] = data["PrimaryOutcomeMeasure"].astype(str)
data["PrimaryTimeFrame"] = data["PrimaryTimeFrame"].astype(str)
data["SecondaryOutcomeMeasure"] = data["SecondaryOutcomeMeasure"].astype(str)
data["SecondaryTimeFrame"] = data["SecondaryTimeFrame"].astype(str)

In [5]:
data_result_df = pd.DataFrame(columns=["NCTId", "Condition", 'Change', "Reference", "Variable","Timepoint"])

In [6]:
data_result_df

,NCTId,Condition,Change,Reference,Variable,Timepoint


In [132]:
# Iteration through data 
# TODO: Remove Measurment-Title ???



def data_to_dataframe(data):
    dfTest = pd.DataFrame(columns=["NCTId","Condition","Change","Reference", "Variable", "Timepoint"])

    for index, row in data.iterrows():
        pointer = 0
        trial_id = row[1]

        condition = row[2]
        
        primary_endpoint = row[3]
        primary_endpoints = []

        primary_endpoint_timepoint = row[4]
        primary_endpoint_timepoints = []

        secondary_endpoint = row[5]
        secondary_endpoints = []

        secondary_endpoint_timepoint = row[6]
        secondary_endpoint_timepoints = []
        
        if  primary_endpoint.find("|") != -1:
            primary_endpoints = primary_endpoint.split("|")

        if  primary_endpoint_timepoint.find("|") != -1:
            primary_endpoint_timepoints = primary_endpoint_timepoint.split("|")

        if  secondary_endpoint.find("|") != -1:
            secondary_endpoints = secondary_endpoint.split('|')

        if  secondary_endpoint_timepoint.find("|") != -1:
            secondary_endpoint_timepoints = secondary_endpoint_timepoint.split('|')
        
        if primary_endpoints == []:
            endpoint_art = "Primary_endpoint"
            dfTemp = process_text_entry(trial_id, condition, primary_endpoint, endpoint_art, pointer)
            dfTest = dfTest.append(dfTemp, ignore_index= True)
        else:
            for point in primary_endpoints:
                endpoint_art = "Primary_endpoint"
                dfTemp = process_text_entry(trial_id, condition, point, endpoint_art, pointer)
                dfTest = dfTest.append(dfTemp, ignore_index= True)


        if secondary_endpoints == []:
            endpoint_art="Secondary_endpoint"
            dfTemp = process_text_entry(trial_id, condition, secondary_endpoint, endpoint_art, pointer)
            dfTest = dfTest.append(dfTemp, ignore_index= True)
        else:
            for point in secondary_endpoints:
                endpoint_art = "Secondary_endpoint"
                dfTemp = process_text_entry(trial_id, condition, point,endpoint_art, pointer)
                dfTest = dfTest.append(dfTemp, ignore_index= True)
                pointer+=1

    return dfTest


In [128]:
def process_text_entry(nctid, condition, text, endpoint_art, pointer):
    dfTemp = pd.DataFrame(columns=["NCTId","Condition","Change","Reference", "Variable", "Timepoint", "Endpoint Art"])

    target = nlp(text)
    for entity in target.ents:
        dfTemp.at[pointer,"NCTId"]=nctid
        dfTemp.at[pointer,"Condition"]=condition
        if entity.label_ != "Condition":
            dfTemp[entity.label_]=entity.text
        dfTemp.at[pointer,"Endpoint Art"]=endpoint_art
    return dfTemp


In [133]:

results_df = data_to_dataframe(data)

In [134]:
results_df

,NCTId,Condition,Change,Reference,Variable,Timepoint,Endpoint Art
0,NCT01814787,Type 2 Diabetes,Number of Children,NaN,Documented Risk Factors,NaN,Primary_endpoint
1,NCT01346033,Type 2 Diabetes,NaN,Validation,SCOUT DS algorithm,NaN,Primary_endpoint
2,NCT04765631,Type 2 Diabetes,outcome,comparison,total volumetric bone mineral density (vBMD),NaN,Primary_endpoint
3,NCT03979768,Type2 Diabetes,Number of Participants,NaN,NaN,NaN,Primary_endpoint
4,NCT03979768,Type2 Diabetes,number of participants,NaN,HbA1c-measurement (HbA1c-group),NaN,Secondary_endpoint
...,...,...,...,...,...,...,...
321,NCT01755468,Type 2 Diabetes,NaN,Baseline,NaN,2-years,Secondary_endpoint
322,NCT04363710,Type 2 Diabetes Mellitus in Obese,NaN,percentage,HbA1c,NaN,Primary_endpoint
323,NCT04363710,Type 2 Diabetes Mellitus in Obese,NaN,liter,Fasting Insulin Level,NaN,Primary_endpoint
324,NCT04363710,Type 2 Diabetes Mellitus in Obese,NaN,BMI,Weight,NaN,Primary_endpoint


In [137]:
results_df.index

RangeIndex(start=0, stop=326, step=1)

In [139]:
export = results_df
export = export.fillna('Not Mentioned')
export.rename_axis("Index", axis='index', inplace=True)
export.to_csv("results.csv")